In [ ]:
# ✅ Install Google Earth Engine and Geemap
!pip install geemap
!pip install earthengine-api
# Install required libraries (run once)
!pip install --upgrade earthengine-api geemap rasterio
!pip install streamlit
!pip install localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [ ]:
# ✅ Import Required Libraries
import ee
import geemap
import streamlit as st
import folium
import math
import numpy as np
import rasterio
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import folium  # Optional if using geemap
import PIL.Image
import urllib.request
import ipywidgets as widgets
from datetime import date
from IPython.display import display, Javascript

# ✅ Authenticate if needed
try:
    ee.Initialize()
    print("✅ Earth Engine initialized!")
except Exception:
    print("🔄 Authenticating to Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project='ee-shahabeddinj')
    print("✅ Earth Engine initialized successfully!")

🔄 Authenticating to Earth Engine...
✅ Earth Engine initialized successfully!


In [ ]:
import ee
import geemap
import math
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import urllib.request
import ipywidgets as widgets
from datetime import date
from IPython.display import display, Javascript


# ✅ Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize(project='ee-shahabeddinj')

# 🌍 Interactive Map for ROI Selection
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.centerObject(ee.Geometry.Point([-72.75, 46.29]), 12)
Map.add_draw_control()

print("🔹 Please **draw** your ROI on the map and click **Submit**.")
display(Map)

# 📆 **Date Selection Widgets**
start_date_widget = widgets.DatePicker(description='Start Date:', value=date(2023, 10, 1), min=date(2015, 1, 1), max=date(2025, 12, 31))
end_date_widget = widgets.DatePicker(description='End Date:', value=date(2024, 6, 30), min=date(2015, 1, 1), max=date(2025, 12, 31))
resolution_widget = widgets.Dropdown(options=[10, 30, 100], value=30, description='Resolution (m):')

roi_button = widgets.Button(description="Submit ROI & Start Processing", button_style='success')
roi_output = widgets.Output()
display(start_date_widget, end_date_widget, resolution_widget, roi_button, roi_output)

# ✅ Cropland Clipping Option
clip_to_agriculture_checkbox = widgets.Checkbox(
    value=False,
    description='Clip to Agricultural Lands Only (Class 15)',
    indent=False
)
display(clip_to_agriculture_checkbox)


# ==========================================
# 📌 **Sentinel-1 Processing**
# ==========================================
def process_sentinel1(start_date, end_date, roi):
    """Loads and processes Sentinel-1 data for the selected ROI and time range."""
    if roi is None:
        print("❌ ERROR: No ROI selected. Please draw an ROI before processing.")
        return None

    # ✅ Extract user-selected resolution
    selected_resolution = resolution_widget.value

    roi = ee.Geometry(roi)

    # ✅ Filter Sentinel-1 Collection
    collection = (
        ee.ImageCollection('COPERNICUS/S1_GRD')
        .filterDate(start_date, end_date)
        .filterBounds(roi)
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
    )

    if collection.size().getInfo() == 0:
        print("❌ ERROR: No Sentinel-1 images found in the selected date range and ROI.")
        return None

    print(f"🔍 Found {collection.size().getInfo()} Sentinel-1 images in ROI.")

    # ✅ Apply Refined Lee Filtering
    def RefinedLee(img):
        img_natural = ee.Image(10.0).pow(img.select('VH').divide(10.0))
        kernel = ee.Kernel.square(7) # Window size
        mean = img_natural.reduceNeighborhood(ee.Reducer.mean(), kernel)
        variance = img_natural.reduceNeighborhood(ee.Reducer.variance(), kernel)
        sample_stats = variance.divide(mean.multiply(mean))
        b = sample_stats.divide(sample_stats.add(1.0))
        refined = mean.add(b.multiply(img_natural.subtract(mean))).float()
        refined_db = refined.log10().multiply(10.0).rename('refined_lee')
        return img.addBands(refined_db)

    # ✅ Apply Terrain Correction
    def normalizeS1(image):
        srtm = ee.Image('USGS/SRTMGL1_003')
        cos2thetaref = ee.Number(40).divide(180).multiply(math.pi).cos().pow(2)
        cos2theta = image.select('angle').divide(180).multiply(math.pi).cos().pow(2)
        vh_corrected = image.select('refined_lee').multiply(cos2thetaref).divide(cos2theta).rename('VH_corrected')
        return image.addBands(vh_corrected)

    processed_collection = (
        collection.map(RefinedLee)
        .map(normalizeS1)
        .map(lambda img: img.reproject(crs="EPSG:4326", scale=selected_resolution))  # ✅ Apply selected resolution
    )

    return processed_collection

# ==========================================
# 📌 **Mosaicking by Date**
# ==========================================
def mosaic_by_date(collection, roi, start_date, end_date):
    """Mosaics Sentinel-1 images captured on the same date to avoid duplicate acquisitions."""
    if collection is None:
        print("❌ ERROR: No processed images available for mosaicking.")
        return None

    filtered_collection = collection.filterDate(start_date, end_date)

    if filtered_collection.size().getInfo() == 0:
        print("❌ ERROR: No images found after filtering for mosaicking.")
        return None

    print(f"📊 Debug: {filtered_collection.size().getInfo()} images before mosaicking.")

    # ✅ Extract unique dates
    unique_dates = filtered_collection.aggregate_array('system:time_start').map(
        lambda millis: ee.Date(millis).format('YYYY-MM-dd')
    ).distinct()

    def mosaic_same_day(date_str):
        date = ee.Date.parse('YYYY-MM-dd', date_str)
        return filtered_collection.filterDate(date, date.advance(1, 'day')).mosaic().clip(roi).set('system:time_start', date.millis())

    mosaicked_collection = ee.ImageCollection(unique_dates.map(mosaic_same_day))

    if mosaicked_collection.size().getInfo() == 0:
        print("❌ ERROR: No mosaicked images generated.")
        return None

    print(f"✅ SUCCESS: Mosaicked {mosaicked_collection.size().getInfo()} images.")
    return mosaicked_collection

# ==========================================
# 📌 **SigmaDiff Computation**
# ==========================================
def compute_sigma_diff_pixelwise(collection):
    """Computes SigmaDiff images step-by-step."""
    if collection is None:
        print("❌ ERROR: No mosaicked images available for SigmaDiff computation.")
        return None

    sorted_collection = collection.sort('system:time_start')
    image_list = sorted_collection.toList(sorted_collection.size())

    first_image = ee.Image(image_list.get(0)).addBands(
        ee.Image.constant(0).float().rename('SigmaDiff')
    )

    history = first_image.select('VH_corrected')
    updated_images = [first_image]

    collection_size = sorted_collection.size().getInfo()

    for i in range(1, collection_size):
        current_img = ee.Image(image_list.get(i))
        sigma_diff = current_img.select('VH_corrected').subtract(history).rename('SigmaDiff')
        updated_images.append(current_img.addBands(sigma_diff))

    return ee.ImageCollection.fromImages(updated_images)


# ==========================================
# 📌 **SigmaDiff Min/Max Computation**
# ==========================================
def compute_sigma_diff_extremes(collection, start_year, user_roi):
    """Computes SigmaDiff_min and SigmaDiff_max dynamically per pixel."""
    if collection is None or collection.size().getInfo() == 0:
        print("❌ ERROR: No valid SigmaDiff images found. Cannot compute extremes.")
        return None

    mid_oct_to_end_jan = collection.filterDate(
        f'{start_year}-10-15', f'{start_year+1}-01-31')

    end_feb_to_may = collection.filterDate(
        f'{start_year+1}-02-20', f'{start_year+1}-05-20')

    sigma_min = mid_oct_to_end_jan.select('SigmaDiff')\
        .reduce(ee.Reducer.min())\
        .rename('SigmaDiff_min')\
        .clip(user_roi)

    sigma_max = end_feb_to_may.select('SigmaDiff')\
        .reduce(ee.Reducer.max())\
        .rename('SigmaDiff_max')\
        .clip(user_roi)

    def attach_min_max(img):
        return img.addBands(sigma_min).addBands(sigma_max)

    updated_collection = collection.map(attach_min_max)
    return updated_collection

# ==========================================
# 📌 **Freeze-Thaw K Assignment**
# ==========================================
def assign_freeze_thaw_k(collection):
    """Assigns Freeze-Thaw K values based on cumulative tracking."""
    sorted_collection = collection.sort('system:time_start')
    image_list = sorted_collection.toList(sorted_collection.size())

    first_image = ee.Image(image_list.get(0)).addBands(
        ee.Image.constant(1).byte().rename('K')
    )

    freeze_tracker = first_image.select('K')
    updated_images = [first_image]

    collection_size = image_list.size().getInfo()

    for i in range(1, collection_size):
        current_img = ee.Image(image_list.get(i))
        sigma_diff = current_img.select('SigmaDiff')
        sigma_min = current_img.select('SigmaDiff_min')
        sigma_max = current_img.select('SigmaDiff_max')

        tolerance = ee.Image(0.01)
        freeze_start = sigma_diff.subtract(sigma_min).abs().lt(tolerance)
        thaw_start = sigma_diff.subtract(sigma_max).abs().lt(tolerance)

        k = freeze_tracker.where(freeze_start, 0).where(thaw_start, 1).byte()
        freeze_tracker = k

        k = k.updateMask(sigma_diff.mask())  # Ensure valid regions only
        updated_img = current_img.addBands(k)
        updated_images.append(updated_img)

    # ✅ Debug: Print the first image's bands
    first_k_image = ee.Image(updated_images[0])
    print(f"📢 Debug: First Image Bands After K Computation: {first_k_image.bandNames().getInfo()}")

    return ee.ImageCollection.fromImages(updated_images)

# ==========================================
# 📌 Step 6: ThawRef Calculation
# ==========================================
def compute_thaw_ref_pixelwise(collection, start_year, user_roi):
    """
    Computes ThawRef for each pixel as the average of the three highest VH_corrected values
    from combined Fall and Spring periods within the selected ROI.

    Args:
        collection (ee.ImageCollection): ImageCollection with 'VH_corrected' band.
        start_year (int): Starting year for the analysis.
        user_roi (ee.Geometry): The user-selected ROI.

    Returns:
        ee.Image: An image where each pixel's value is the computed ThawRef.
    """
    # Define the date ranges for Fall and Spring periods
    fall_start = f'{start_year}-10-01'
    fall_end = f'{start_year}-11-30'
    spring_start = f'{start_year+1}-04-15'
    spring_end = f'{start_year+1}-06-10'

    # Filter the collection for Fall and Spring periods
    fall_collection = collection.filterDate(fall_start, fall_end)
    spring_collection = collection.filterDate(spring_start, spring_end)

    # Combine Fall and Spring collections
    combined_collection = fall_collection.merge(spring_collection)

    # Clip images to the user-defined ROI
    combined_collection = combined_collection.map(lambda img: img.clip(user_roi))

    # Sort the collection by VH_corrected in descending order
    sorted_collection = combined_collection.sort('VH_corrected', False)

    # Limit the collection to the top 3 images per pixel
    top3_collection = sorted_collection.limit(3)

    # Compute the mean of the top 3 VH_corrected values per pixel
    thaw_ref_image = top3_collection.mean().select('VH_corrected').rename('ThawRef')

    return thaw_ref_image

# ==========================================
# 📌 Step 7: DeltaTheta (ΔΘ) Calculation
# ==========================================
# ✅ Generalized function to compute DeltaTheta (ΔΘ)
def compute_delta_theta(collection, thaw_ref_image):
    """
    Computes ΔΘ (DeltaTheta) as ThawRef - VH_corrected for each image dynamically.

    Inputs:
        collection: ImageCollection with VH_corrected band.
        thaw_ref_image: ThawRef Image (constant band computed previously).

    Outputs:
        ImageCollection with an added DeltaTheta band per image.
    """

    def add_delta_theta(img):
        vh_corrected = img.select('VH_corrected')

        # ✅ Compute ΔΘ = ThawRef - VH_corrected
        delta_theta = thaw_ref_image.subtract(vh_corrected).rename('DeltaTheta')

        # ✅ Ensure correct masking (mask ΔΘ where VH_corrected is No Data)
        delta_theta = delta_theta.updateMask(vh_corrected.mask())

        return img.addBands(delta_theta)

    return collection.map(add_delta_theta)


# ==========================================
# 📌 Step 8: EFTA Calculation
# ==========================================
# ✅ Debug: Check Mean Values Before Applying EFTA
selected_resolution = resolution_widget.value  # ✅ Get user-selected resolution


# ✅ Compute EFTA Dynamically

def compute_efta(collection):
    """
    Computes EFTA dynamically using the exponential freeze-thaw algorithm.
    Inputs:
        collection: ImageCollection with necessary bands (K, ThawRef, VH_corrected, DeltaTheta).
    Outputs:
        ImageCollection with EFTA band added per image.
    """
    selected_resolution = resolution_widget.value  # ✅ Extract user-selected resolution

    def calculate_efta(img):
        # ✅ Retrieve Required Bands
        k = img.select('K')
        thaw_ref = img.select('ThawRef')
        vh_corrected = img.select('VH_corrected')
        delta_theta = img.select('DeltaTheta')

        # ✅ Compute the Exponential Component
        exp_component = (ee.Image(1)
                         .add(thaw_ref.divide(vh_corrected))
                         .multiply(k.multiply(-1))
                         .exp())

        # ✅ Compute EFTA
        efta = exp_component.multiply(delta_theta).rename('EFTA')

        # ✅ Ensure proper masking (Handle No Data)
        efta = efta.updateMask(vh_corrected.mask())

        return img.addBands(efta).reproject(crs="EPSG:4326", scale=selected_resolution)  # ✅ Apply resolution

    return collection.map(calculate_efta)


# ==========================================
# 📌 **Step 9: Freeze-Thaw Classification Using RF**
# ==========================================
# ✅ Import training data from GEE asset
training_asset = ee.FeatureCollection('projects/ee-shahabeddinj/assets/training_data')

# ✅ Define bands and label
bands = ['EFTA']  # Feature(s) used for classification
label = 'label'   # Class label (0 = Frozen, 1 = Thawed)

# ✅ Print sample training data
sample_training = training_asset.limit(5).getInfo()
print(f"✅ Sample Training Data:\n{sample_training}")

# ✅ Train the RF model in GEE
rf_model = ee.Classifier.smileRandomForest(
    numberOfTrees=150,
    variablesPerSplit=1,
    minLeafPopulation=3,
    seed=42
).train(
    features=training_asset,
    classProperty=label,
    inputProperties=bands
)

print("✅ RF model trained successfully in GEE.")

# ✅ Classify images in efta_collection using the trained RF model
def classify_image(img):
    selected_resolution = resolution_widget.value  # ✅ Extract user-selected resolution
    classified = img.select('EFTA').classify(rf_model).rename('FT_State')
    return img.addBands(classified).reproject(crs="EPSG:4326", scale=selected_resolution)  # ✅ Apply resolution



# ==========================================
# 📌 **Step 1: Ensure ROI Selection Before Processing**
# ==========================================
def submit_roi(b):
    """
    Handles the entire pipeline from ROI selection to Freeze-Thaw classification.
    """
    global user_roi, processed_images, mosaicked_images, sigma_diff_collection
    global sigma_extreme_collection, final_k_collection, thaw_ref_collection, thaw_ref_image
    global delta_theta_collection, efta_collection, classified_collection

    with roi_output:
        roi_output.clear_output()

        # ✅ Ensure the user has drawn an ROI before proceeding
        if Map.user_roi is None:
            print("❌ ERROR: No ROI selected. Please draw an ROI before processing.")
            return


        # ✅ Extract user-selected ROI
        user_roi = ee.Geometry(Map.user_roi)

        # ✅ Optional Cropland Masking (Clip to Agriculture Only)
        if clip_to_agriculture_checkbox.value:
            print("🌱 Cropland-only mode enabled. Clipping ROI to agricultural areas (Class 15)...")

        # ✅ Load North America 2020 Land Cover (NALCMS)
        landcover = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS").select("landcover")

        # ✅ Create a mask for cropland only (class 15)
        cropland_mask = landcover.eq(15)

        # ✅ Extract cropland geometry within the user ROI
        cropland_geometry = cropland_mask.selfMask().reduceToVectors(
            geometry=user_roi,
            geometryType='polygon',
            scale=30,
            maxPixels=1e13
        )

        # ✅ Intersect user's ROI with cropland polygons
        user_roi = user_roi.intersection(cropland_geometry.geometry(), ee.ErrorMargin(30))

        # ✅ Check if ROI still contains valid cropland area
        empty_check = user_roi.coordinates().size().getInfo()
        if empty_check == 0:
           print("❌ ERROR: Cropland mask removed entire ROI. Please select a different area or disable cropland-only mode.")
           return



        user_selected_start = start_date_widget.value.strftime("%Y-%m-%d")
        user_selected_end = end_date_widget.value.strftime("%Y-%m-%d")
        today = date.today().strftime("%Y-%m-%d")  # Get today's date as string


        # ✅ Prevent user from selecting a future date beyond today
        if user_selected_end >= today:
            print(f"❌ ERROR: Selected end date ({user_selected_end}) is in the future. Please choose a date before {today}.")
            return

        # ✅ Ensure start_date is before end_date
        if user_selected_start >= user_selected_end:
            print(f"❌ ERROR: Start date ({user_selected_start}) must be earlier than end date ({user_selected_end}).")
            return

        # ✅ Determine Processing Window (Always October to June)
        start_year = int(user_selected_start[:4])

        # Rule: If start date is between October 1st and December 31st → Keep `start_year`
        # Rule: If start date is between January 1st and September 30th → Subtract 1 from `start_year`
        if int(user_selected_start[5:7]) < 10:  # If month is January (01) to September (09)
            start_year -= 1

        start_date = f"{start_year}-10-01"
        end_date = f"{start_year+1}-06-30"

        # ✅ Handle Future Data Gaps
        # If the user-selected period extends beyond available data (today), limit visualization
        if end_date >= today:
            print(f"⚠️ WARNING: Some future dates are unavailable. Adjusting processing to available data.")
            end_date = today  # Clip the processing end to today

        print(f"✅ User Selected Inputs:\n - ROI: Defined\n - Start: {user_selected_start}\n - End: {user_selected_end}")
        print(f"🔹 Adjusted Processing Range: {start_date} to {end_date}")

        # ✅ Process Sentinel-1 Data
        processed_images = process_sentinel1(start_date, end_date, user_roi)
        if processed_images is None:
            return

        # ✅ Apply Mosaicking
        mosaicked_images = mosaic_by_date(processed_images, user_roi, start_date, end_date)
        if mosaicked_images is None:
            return

        # ✅ Compute SigmaDiff (No Visualization)
        sigma_diff_collection = compute_sigma_diff_pixelwise(mosaicked_images)
        if sigma_diff_collection is None:
            return

        # ✅ Compute SigmaDiff Min/Max (No Visualization)
        sigma_extreme_collection = compute_sigma_diff_extremes(sigma_diff_collection, start_year, user_roi)
        if sigma_extreme_collection is None:
            return

        # ✅ Compute Freeze-Thaw K
        final_k_collection = assign_freeze_thaw_k(sigma_extreme_collection)
        if final_k_collection is None:
            print("❌ ERROR: K computation failed. Stopping execution.")
            return

        print("✅ SUCCESS: Freeze-Thaw K Computed.")

        # ✅ Compute ThawRef
        thaw_ref_image = compute_thaw_ref_pixelwise(final_k_collection, start_year, user_roi)
        if thaw_ref_image is None:
            return

        # ✅ Add ThawRef Band to Each Image
        thaw_ref_collection = final_k_collection.map(lambda img: img.addBands(thaw_ref_image))

        # ✅ Compute DeltaTheta
        delta_theta_collection = compute_delta_theta(thaw_ref_collection, thaw_ref_image)
        if delta_theta_collection is None:
            print("❌ ERROR: DeltaTheta computation failed. Stopping execution.")
            return

        print("✅ SUCCESS: DeltaTheta Computed.")

        # ✅ Compute EFTA
        efta_collection = compute_efta(delta_theta_collection)
        if efta_collection is None:
            print("❌ ERROR: EFTA computation failed. Stopping execution.")
            return

        print("✅ SUCCESS: EFTA Computed.")

        # ✅ Import training data from GEE asset
        training_asset = ee.FeatureCollection('projects/ee-shahabeddinj/assets/training_data')

        # ✅ Train the RF model in GEE
        rf_model = ee.Classifier.smileRandomForest(
            numberOfTrees=150,
            variablesPerSplit=1,
            minLeafPopulation=3,
            seed=42
        ).train(
            features=training_asset,
            classProperty='label',
            inputProperties=['EFTA']
        )

        print("✅ RF model trained successfully in GEE.")

        # ✅ Classify images in efta_collection using the trained RF model
        classified_collection = efta_collection.map(lambda img: img.addBands(
            img.select('EFTA').classify(rf_model).rename('FT_State')
        ))

        # ✅ Ensure Visualization Uses Only User-Selected Date Range
        classified_collection_visual = classified_collection.filterDate(user_selected_start, user_selected_end)

        # ✅ Visualize Freeze-Thaw Classification (Only User-Selected Date Range)
        visualize_ft_classification(classified_collection_visual, user_roi, resolution_widget.value)


        print("✅ All Processing Completed.")



# ==========================================
# 📌 **Step 2: Compute and Summarize FT Classification**
# ==========================================
def summarize_ft_classification(collection, user_roi):
    """
    Computes statistical summary of Freeze-Thaw classification (Frozen vs. Thawed pixels)
    for each image in the collection.
    """
    selected_resolution = resolution_widget.value  # ✅ Extract user-selected resolution

    if collection.size().getInfo() == 0:
        print("❌ No Freeze-Thaw classified images available for summarization.")
        return

    image_list = collection.toList(collection.size())
    num_images = collection.size().getInfo()

    print("\n📊 **Freeze-Thaw Classification Summary**:\n")

    for i in range(num_images):
        img = ee.Image(image_list.get(i))

        # Extract date
        timestamp = img.get("system:time_start").getInfo()
        date_str = ee.Date(timestamp).format("YYYY-MM-dd").getInfo()

        # ✅ Apply user-selected resolution dynamically
        stats = img.select("FT_State").reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=user_roi,
            scale=selected_resolution,  # ✅ Apply selected resolution
            maxPixels=1e13
        ).getInfo()

        hist = stats.get("FT_State", {})

        thawed_count = int(hist.get("0", 0))
        frozen_count = int(hist.get("1", 0))
        total_count = thawed_count + frozen_count
        thawed_percent = (thawed_count / total_count * 100) if total_count > 0 else 0
        frozen_percent = (frozen_count / total_count * 100) if total_count > 0 else 0

        print(f"📅 Image {i+1} (Date: {date_str}):")
        print(f"   - **Thawed Pixels (0):** {thawed_count} ({thawed_percent:.2f}%)")
        print(f"   - **Frozen Pixels (1):** {frozen_count} ({frozen_percent:.2f}%)")
        print("--------------------------------------------------")

    print("✅ Freeze-Thaw Classification Summary Computed Successfully.")


# ==========================================
# 📌 **Step 3: Visualize FT Classification**
# ==========================================
def visualize_ft_classification(collection, user_roi, resolution):
    """
    Visualizes Freeze-Thaw classification images with a single-row legend.
    """
    if collection.size().getInfo() == 0:
        print("❌ No Freeze-Thaw classification images available for visualization.")
        return

    image_list = collection.toList(collection.size())
    num_images = collection.size().getInfo()

    print(f"📊 Total Freeze-Thaw Classified Images Available for Visualization: {num_images}")

    cols = 3  # Three columns for images
    rows = (num_images // cols) + (num_images % cols > 0)  # Calculate required rows

    # If the last row is full, legend goes in a new row, else it fits after the last image
    total_slots = rows * cols
    legend_needed = total_slots > num_images
    if not legend_needed:
        rows += 1  # Add a row for the legend if needed

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()

    summary_lines = []  # To store compact pixel summaries

    for i in range(num_images):
        img = ee.Image(image_list.get(i))

        # Extract the date for reference
        timestamp = img.get("system:time_start").getInfo()
        date_str = ee.Date(timestamp).format("YYYY/MM/dd").getInfo()

        # Get the thumbnail URL for FT_State
        url = img.select("FT_State").clip(user_roi).getThumbURL({
            "min": 0, "max": 1, "dimensions": 512, "palette": ["red", "blue"]
        })

        # Load image and plot
        image_array = np.array(PIL.Image.open(urllib.request.urlopen(url)))
        axes[i].imshow(image_array, cmap="bwr", vmin=0, vmax=1)
        axes[i].set_title(f"{date_str}")
        axes[i].axis("off")  # Remove grids

        # ✅ Compute pixel statistics
        stats = img.select("FT_State").reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=user_roi,
            scale=resolution,  # Use user-selected resolution
            maxPixels=1e13
        ).getInfo()

        hist = stats.get("FT_State", {})
        thawed_count = int(hist.get("0", 0))
        frozen_count = int(hist.get("1", 0))
        total_count = thawed_count + frozen_count
        thawed_percent = (thawed_count / total_count * 100) if total_count > 0 else 0
        frozen_percent = (frozen_count / total_count * 100) if total_count > 0 else 0

        # ✅ Store compact summary line
        summary_lines.append(f"{date_str}: Frozen={frozen_count} ({frozen_percent:.1f}%) | Thawed={thawed_count} ({thawed_percent:.1f}%)")

    # ✅ Place legend in the last available space
    if legend_needed:
        legend_ax = axes[num_images]  # Place legend in the first available empty slot
    else:
        legend_ax = fig.add_subplot(rows, cols, num_images + 1)  # New row for the legend

    legend_ax.axis("off")  # Remove axis

    # ✅ Create legend colors and labels in a single row
    legend_ax.add_patch(plt.Rectangle((0, 0), 0.1, 0.1, fc="red", label="**Thawed**"))
    legend_ax.add_patch(plt.Rectangle((0.2, 0), 0.1, 0.1, fc="blue", label="**Frozen**"))

    legend_ax.legend(loc="center", ncol=2, frameon=False, fontsize=10)  # Single row

    plt.tight_layout()
    plt.show()

    # ✅ Print compact pixel summary for each image
    print(f"\n📊 **Freeze-Thaw Pixel Summary with resolution of {resolution} meters**:")
    for line in summary_lines:
        print(line)

    print("✅ Freeze-Thaw Classification Visualization Complete.")





# ✅ Attach the function to the button
roi_button.on_click(submit_roi)



🔹 Please **draw** your ROI on the map and click **Submit**.


Map(center=[46.29, -72.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

DatePicker(value=datetime.date(2023, 10, 1), description='Start Date:')

DatePicker(value=datetime.date(2024, 6, 30), description='End Date:')

Dropdown(description='Resolution (m):', index=1, options=(10, 30, 100), value=30)

Button(button_style='success', description='Submit ROI & Start Processing', style=ButtonStyle())

Output()

Checkbox(value=False, description='Clip to Agricultural Lands Only (Class 15)', indent=False)

✅ Sample Training Data:
{'type': 'FeatureCollection', 'columns': {'EFTA': 'Float', 'label': 'Integer', 'system:index': 'String'}, 'version': 1740416816477192, 'id': 'projects/ee-shahabeddinj/assets/training_data', 'properties': {'system:asset_size': 229066}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '00000000000000000001', 'properties': {'EFTA': 0.5498872043870915, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '00000000000000000003', 'properties': {'EFTA': 0.1586705580772727, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '00000000000000000005', 'properties': {'EFTA': 5.894274166666666, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '00000000000000000006', 'properties': {'EFTA': 0.7842271242553975, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': 